In [1]:
#|export
import subprocess
import dspy
import logging

In [2]:
import ollama


In [3]:
# |export
########## DSPy Config  ##########
llm = dspy.OllamaLocal("nous-llama-3-8b-instruct", max_tokens=4000, model_type="text")
dspy.settings.configure(lm=llm)

In [4]:
llm("Hi, how are you?")

["I'm doing well, thank you for asking! I'm a large language model, so I don't have emotions like humans do, but I'm always happy to help with any questions or tasks you may have. How can I assist you today?"]

In [5]:
########## Arize Phoenix ##########
import phoenix as px
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [6]:
phoenix_session = px.launch_app()
endpoint = "http://localhost:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [7]:
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()
class TestTrace(dspy.Signature):
    """
    Perform the task requested to the best of your ability.
    """

    task = dspy.InputField(desc="Task to be performed.")
    answer = dspy.OutputField(desc="The answer.")

In [8]:
test_trace = dspy.Predict(TestTrace)
answer = test_trace(task="Say Hello.")
test_trace_cot = dspy.ChainOfThought(TestTrace)
test_trace_cot(task="Write a python function that adds two numbers together.")

Prediction(
    rationale="---\n\nTask: Write a Python function that adds two numbers together.\nReasoning: Let's think step by step in order to create a simple and efficient function. We will define the function with two parameters, `num1` and `num2`, which are the numbers we want to add. Then, we will use the built-in `+` operator to perform the addition and return the result.",
    answer='```\ndef add_numbers(num1, num2):\n    return num1 + num2\n```'
)

In [10]:
cladio_binary = test_trace_cot(task="Write a short story about a cat named Claudio that quickly devolves into just python code and then just binary.")

In [15]:
cladio_binary.answer

'The story has devolved into pure Python code, which has then been converted into binary.'

In [ ]:
test